In [1]:
%load_ext autoreload
%load_ext autotime

time: 178 µs (started: 2024-10-09 16:27:40 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 541 µs (started: 2024-10-09 16:27:40 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 758 µs (started: 2024-10-09 16:27:40 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-resnet50-dlio-v100-node-16-v3"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/resnet50-dlio-v100-node-16-v3"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/resnet50/dlio-v100/node-16/v3/COMPACT/*.pfw.gz"


time: 645 µs (started: 2024-10-09 16:27:40 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 6.36 s (started: 2024-10-09 16:27:40 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 3min 5s (started: 2024-10-09 16:27:47 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          202.68 seconds                                                                                │
│  I/O Time         2.56 seconds                                                                                  │
│                   ├── Read - 1.41 seconds (55.26%)                                                              │
│                   ├── Write - 0.00 seconds (0.08%)                                                              │
│                   └── Metadata - 2.33 seconds (90.89%)                                                          │
│  I/O Operations   11,640,839 ops                                                                                │
│                   ├── Read - 3,214,719 ops (27.62%)                                                             │
│                   ├── Write - 4,022 ops (0.03%)                                                                 │
│                   └── Metadata - 8,386,770 ops (72.05%)                                                         │
│  I/O Size         727.93 GiB                                                                                    │
│                   ├── Read - 727.93 GiB (100.00%)                                                               │
│                   └── Write - 179.65 kiB (0.00%)                                                                │
│  Read Requests    4 kiB-256 kiB - 3,214,136 ops                                                                 │
│                   ├── <4 kiB - 175,232 ops (5.45%)                                                              │
│                   ├── 4-16 kiB - 20,032 ops (0.62%)                                                             │
│                   ├── 16-64 kiB - 31,353 ops (0.98%)                                                            │
│                   └── 64-256 kiB - 2,987,519 ops (92.95%)                                                       │
│  Write Requests   4 kiB-4 kiB - 4,022 ops                                                                       │
│                   └── <4 kiB - 4,022 ops (100.00%)                                                              │
│  Nodes            16 nodes                                                                                      │
│                   ├── 11194065546293973417 - 2.56 s (100.00%) - 45.50 GiB/10.56 kiB R/W (6.25/0.00%) - 727,549  │
│                   │   ops (6.25%)                                                                               │
│                   ├── 784840397337378243 - 2.54 s (99.38%) - 45.50 GiB/10.56 kiB R/W (6.25/0.00%) - 727,548     │
│                   │   ops (6.25%)                                                                               │
│                   ├── 13943161580101246997 - 2.52 s (98.25%) - 45.50 GiB/10.56 kiB R/W (6.25/0.00%) - 727,559   │
│                   │   ops (6.25%)                                                                               │
│                   ├── 12775585946864612109 - 2.49 s (97.46%) - 45.50 GiB/21.21 kiB R/W (6.25/0.00%) - 727,609   │
│                   │   ops (6.25%)                                                                               │
│                   ├── 17424505939907203644 - 2.46 s (96.29%) - 45.50 GiB/10.56 kiB R/W (6.25/0.00%) - 727,547   │
│                   │   ops (6.25%)                                                                               │
│                   ├── 3331529830307220231 - 2.46 s (96.17%) - 45.50 GiB/10.56 kiB R/W (6.25/0.00%) - 727,552    │
│                   │   ops (6.25%)                                                                               │
│                   ├── 3903101653980564509 - 2.46 s (96

time: 5.67 s (started: 2024-10-09 16:30:52 -07:00)
